In [ ]:
import warnings, os
import datetime as dt
import itertools as it
from numpy import nan, inf
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Storage/Raw.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$'], TFRAMES=['H1'])
Raw.head()

# Data

In [ ]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src['A']        = Query['a']
    Src['Z']        = Query['z']
    Src['Symbol']   = Query['symbol']
    Src['TF']       = Query['tf']
    
    Src['Datetime'] = Query['datetime']
    Src['Date']     = Query['datetime'].dt.date
    Src['Time']     = Query['datetime'].dt.time
    
    Src['Ticks']    = Query['tick_volume']
    Src['Volume']   = Query['real_volume']
    Src['Price']    = Query['close']
    
    Src['Open']     = Query['open']
    Src['High']     = Query['high']
    Src['Low']      = Query['low']
    Src['Close']    = Query['close']

    Src['Change']       = Src['Close'] - Src['Open']
    Src['Hilo']         = Src['High']  - Src['Low']

    Src['Sign']         = Src['Change'].apply(lambda x: -1 if (x <  0) else  +1)
    Src['Sign Bull']    = Src['Change'].apply(lambda x: +1 if (x >= 0) else nan)
    Src['Sign Bear']    = Src['Change'].apply(lambda x: -1 if (x <  0) else nan)
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['Chg Abs'] = Calc['Change'].abs()
    Calc['Chg Pos'] = Calc['Chg Abs'] * Calc['Sign Bull']
    Calc['Chg Neg'] = Calc['Chg Abs'] * Calc['Sign Bear']

    Calc['HL Pos']  = Calc['Hilo'] * Calc['Sign Bull']
    Calc['HL Neg']  = Calc['Hilo'] * Calc['Sign Bear']
    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

# Stats

In [19]:
Calc.groupby('Time')[['Change']].describe().round(0).astype(int)

Change                                    
          count mean  std   min  25% 50%  75%   max
Time                                               
09:00:00   1241   -6  608 -3113 -377  16  351  2811
10:00:00   1241   20  777 -2665 -454 -11  488  5561
11:00:00   1241  -41  648 -3204 -431 -17  331  3410
12:00:00   1241   14  547 -2831 -270  20  302  2270
13:00:00   1246    4  412 -1803 -210   0  232  1551
14:00:00   1245   -3  465 -3291 -239   0  234  4059
15:00:00   1245  -10  491 -4266 -223  -6  242  2745
16:00:00   1245    4  507 -2659 -198  14  225  5505
17:00:00   1245  -18  402 -2428 -200  -5  183  1795
18:00:00    676   19  285 -2530 -130  18  145  1121

In [20]:
Calc.groupby('Time')[['Chg Abs']].describe().round(0).astype(int)

Chg Abs                                   
           count mean  std min  25%  50%  75%   max
Time                                               
09:00:00    1241  460  398   0  163  361  647  3113
10:00:00    1241  593  502   0  221  472  826  5561
11:00:00    1241  488  428   0  185  384  670  3410
12:00:00    1241  399  373   0  141  287  528  2831
13:00:00    1246  300  282   0   95  222  412  1803
14:00:00    1245  320  337   0   95  237  417  4059
15:00:00    1245  332  362   0   92  227  433  4266
16:00:00    1245  331  383   0  100  212  434  5505
17:00:00    1245  280  289   0   88  193  363  2428
18:00:00     676  197  207   0   66  140  267  2530

In [21]:
Calc.groupby('Time')[['Chg Pos']].describe().round(0).astype(int)

Chg Pos                                   
           count mean  std min  25%  50%  75%   max
Time                                               
09:00:00     638  442  387   0  138  344  649  2811
10:00:00     617  617  526   0  242  495  886  5561
11:00:00     608  456  406   0  177  340  633  3410
12:00:00     638  402  370   0  146  290  545  2270
13:00:00     624  303  272   0  102  232  417  1551
14:00:00     636  310  343   0   90  230  409  4059
15:00:00     617  324  336   0   87  247  437  2745
16:00:00     650  321  403   0  100  211  400  5505
17:00:00     620  263  262   0   88  184  345  1795
18:00:00     365  201  193   0   68  134  297  1121

In [22]:
Calc.groupby('Time')[['Chg Neg']].describe().round(0).astype(int)

Chg Neg                                   
           count mean  std   min  25%  50%  75% max
Time                                               
09:00:00     603 -479  408 -3113 -644 -383 -187  -5
10:00:00     624 -570  476 -2665 -788 -444 -209  -5
11:00:00     633 -519  447 -3204 -696 -420 -197  -5
12:00:00     603 -396  378 -2831 -520 -277 -140  -6
13:00:00     622 -297  292 -1803 -408 -210  -90  -6
14:00:00     609 -330  331 -3291 -425 -244 -102  -5
15:00:00     628 -339  386 -4266 -428 -220  -99  -5
16:00:00     595 -342  360 -2659 -464 -217 -103  -5
17:00:00     625 -297  312 -2428 -382 -200  -89  -5
18:00:00     311 -194  222 -2530 -256 -142  -64  -5

In [23]:
Calc.groupby('Time')[['Hilo']].describe().round(0).astype(int)

Hilo                                       
         count  mean  std  min  25%   50%   75%   max
Time                                                 
09:00:00  1241   943  437  241  630   858  1143  3852
10:00:00  1241  1166  511  279  805  1064  1428  6210
11:00:00  1241   992  472  279  654   905  1196  3765
12:00:00  1241   802  433  197  488   699   997  3259
13:00:00  1246   655  352  143  403   570   824  3176
14:00:00  1245   681  429  161  410   576   813  6275
15:00:00  1245   700  463  133  402   572   855  5016
16:00:00  1245   674  501  114  382   544   809  7753
17:00:00  1245   561  375  101  319   466   661  3001
18:00:00   676   362  248   75  204   292   448  2811

In [24]:
Calc.groupby('Time')[['HL Pos']].describe().round(0).astype(int)

HL Pos                                       
          count  mean  std  min  25%   50%   75%   max
Time                                                  
09:00:00    638   936  422  241  626   861  1143  3236
10:00:00    617  1193  526  332  835  1111  1442  6210
11:00:00    608   957  449  279  634   864  1164  3722
12:00:00    638   798  426  197  485   704   990  2982
13:00:00    624   654  344  143  412   575   811  3176
14:00:00    636   661  448  166  393   554   780  6275
15:00:00    617   692  462  133  405   586   830  4095
16:00:00    650   669  555  152  384   534   772  7753
17:00:00    620   532  318  120  318   450   646  2361
18:00:00    365   366  228   75  197   298   475  1388

In [25]:
Calc.groupby('Time')[['HL Neg']].describe().round(0).astype(int)

HL Neg                                       
          count  mean  std   min   25%   50%  75%  max
Time                                                  
09:00:00    603  -950  452 -3852 -1144  -851 -635 -309
10:00:00    624 -1139  496 -3011 -1394 -1028 -770 -279
11:00:00    633 -1026  492 -3765 -1232  -945 -681 -293
12:00:00    603  -807  441 -3259 -1006  -694 -489 -208
13:00:00    622  -657  361 -2356  -832  -565 -392 -174
14:00:00    609  -702  407 -3758  -871  -591 -426 -161
15:00:00    628  -708  465 -5016  -886  -560 -397 -153
16:00:00    595  -681  434 -3326  -832  -560 -379 -114
17:00:00    625  -589  422 -3001  -683  -473 -325 -101
18:00:00    311  -357  270 -2811  -421  -289 -209 -104

# Snippets

In [ ]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src['Time']      = Query['datetime'].dt.time
    Src['Change']    = Src['Close'] - Src['Open']
    Src['Hilo']      = Src['High']  - Src['Low']
    Src['Sign']      = Src['Change'].apply(lambda x: -1 if (x <  0) else  +1)
    Src['Sign Bull'] = Src['Change'].apply(lambda x: +1 if (x >= 0) else nan)
    Src['Sign Bear'] = Src['Change'].apply(lambda x: -1 if (x <  0) else nan)
    return Src


def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['Chg Abs'] = Calc['Change'].abs()
    Calc['Chg Pos'] = Calc['Chg Abs'] * Calc['Sign Bull']
    Calc['Chg Neg'] = Calc['Chg Abs'] * Calc['Sign Bear']

    Calc['HL Pos']  = Calc['Hilo'] * Calc['Sign Bull']
    Calc['HL Neg']  = Calc['Hilo'] * Calc['Sign Bear']
    return Calc


Calc.groupby('Time')[['Hilo']].describe().round(0).astype(int)
Calc.groupby('Time')[['HL Pos']].describe().round(0).astype(int)
Calc.groupby('Time')[['HL Neg']].describe().round(0).astype(int)

Calc.groupby('Time')[['Change']].describe().round(0).astype(int)
Calc.groupby('Time')[['Chg Abs']].describe().round(0).astype(int)
Calc.groupby('Time')[['Chg Pos']].describe().round(0).astype(int)
Calc.groupby('Time')[['Chg Neg']].describe().round(0).astype(int)